In [1]:
import pandas as pd
import numpy as np
import os
import requests

In [2]:
import pyodbc
import urllib
import sqlalchemy
# MySQL connection parameters
db_params = {
    'user': 'han',
    'password': 'joseph',
    'host': '127.0.0.1',  # Usually 'localhost' or an IP address
    'port': '4450',  # Default MySQL port is 3306
    'database': 'ITRAVEL',
}

# URL-encode the password (if it contains special characters)
db_params['password'] = urllib.parse.quote_plus(db_params['password'])

# Create a MySQL database connection using SQLAlchemy
params_mysql = "mysql+mysqlconnector://{user}:{password}@{host}:{port}/{database}".format(**db_params)
engine_mysql = sqlalchemy.create_engine(params_mysql)

In [3]:
engine_p=engine_mysql

In [4]:
pd.set_option('display.max_rows', None)

In [4]:
'''get locations data from source database'''
df_locations = pd.read_sql_table('locations',engine_p)
df_reviews  = pd.read_sql_table('location_reviews',engine_p)
df_users  = pd.read_sql_table('users',engine_p)

In [5]:
#  df_users = df_users.drop('id', axis=1)
df_users.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9834 entries, 0 to 9833
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   username          9834 non-null   object
 1   user_location_id  9834 non-null   int64 
 2   name              9834 non-null   object
 3   avatar_thumbnail  9834 non-null   object
 4   avatar_small      9834 non-null   object
 5   avatar_medium     9834 non-null   object
 6   avatar_large      9834 non-null   object
 7   avatar_original   9834 non-null   object
dtypes: int64(1), object(7)
memory usage: 614.8+ KB


In [5]:
def get_user(df_review):
    user=pd.DataFrame()
    # Loop through the rows using iterrows()
    for index, row in df_review.iterrows():
        data=eval(row['user'])
        
        user_location = data.get('user_location', {})
        user_location_id = user_location.get('id', 0)
        user_location_name = user_location.get('name', 'null')

        avatar = data.get('avatar', {})
        avatar_thumbnail = avatar.get('thumbnail', 'null')
        avatar_small = avatar.get('small', 'null')
        avatar_medium = avatar.get('medium', 'null')
        avatar_large = avatar.get('large', 'null')
        avatar_original = avatar.get('original', 'null')

        df1 = pd.DataFrame({'username': data['username'], 
                            'user_location_id': user_location_id, 
                            'name': user_location_name,
                            'avatar_thumbnail':avatar_thumbnail,
                            'avatar_small':avatar_small,
                            'avatar_medium':avatar_medium,
                            'avatar_large':avatar_large,
                            'avatar_original':avatar_original                       
                           }, index=[0])
        user=pd.concat([user,df1])


    user=user.reset_index(drop=True)

    if 'user_location_id' in user.columns:
        user['user_location_id']=user['user_location_id'].replace('null',0)
    
    return user


In [6]:
# Define a location reviews function
def call_location_reviews(location_id):
    """This function get location reviews through api ."""
    
    # API URL
    api_url = "https://api.content.tripadvisor.com/api/v1/location/" + str(location_id) + "/reviews?language=en&key=72577774A72F48D9BF1180DEA12E312A"

    # Make API request
    response = requests.get(api_url)
#     data = response.json()
#     df = pd.DataFrame(data['data'])
        
    try:
        data = response.json()
        df = pd.DataFrame(data['data'])
    except KeyError:
        print("Key 'data' not found in the JSON response.")
        df = pd.DataFrame() 
        return df
    
        
    if 'user' in df.columns:
        df['user'] = df['user'].astype(str)
    else:
        df['user']='null'

    if 'subratings' in df.columns:
        df['subratings'] = df['subratings'].astype(str)
    else:
        df['subratings']='null'
        
    if 'owner_response' in df.columns:
        df['owner_response'] = df['owner_response'].astype(str)
    else:
        df['owner_response'] = 'null'
    return df

In [7]:
# Loop through DataFrame rows
df_new_reviews=pd.DataFrame()

itv=0
for index, row in df_locations.iterrows():
    if index < (itv+0) or index >= (itv+500) :
        continue;
        
    location_id = row['location_id']
    df1=call_location_reviews(location_id)
    df_new_reviews=pd.concat([df_new_reviews, df1], ignore_index=True)

Key 'data' not found in the JSON response.


In [8]:
df_new_reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2213 entries, 0 to 2212
Data columns (total 15 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   user              2213 non-null   object 
 1   subratings        2213 non-null   object 
 2   owner_response    2213 non-null   object 
 3   id                2213 non-null   float64
 4   lang              2213 non-null   object 
 5   location_id       2213 non-null   float64
 6   published_date    2213 non-null   object 
 7   rating            2213 non-null   float64
 8   helpful_votes     2213 non-null   float64
 9   rating_image_url  2213 non-null   object 
 10  url               2213 non-null   object 
 11  text              2213 non-null   object 
 12  title             2213 non-null   object 
 13  trip_type         2198 non-null   object 
 14  travel_date       2198 non-null   object 
dtypes: float64(4), object(11)
memory usage: 259.5+ KB


In [9]:
# df_new_reviews
df_new_reviews['review_id']=df_new_reviews['id']
df_new_reviews = df_new_reviews.drop('id', axis=1)

'''drop duplicate new review data ''' 
df_new_reviews=df_new_reviews.drop_duplicates(subset='review_id')

'''read existing review data ''' 
df_reviews  = pd.read_sql_table('location_reviews',engine_p)


'''remove existing review records'''
df_new_reviews = df_new_reviews[~df_new_reviews['review_id'].isin(df_reviews['review_id'])]
df_new_reviews.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 854 entries, 0 to 2208
Data columns (total 15 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   user              854 non-null    object 
 1   subratings        854 non-null    object 
 2   owner_response    854 non-null    object 
 3   lang              854 non-null    object 
 4   location_id       854 non-null    float64
 5   published_date    854 non-null    object 
 6   rating            854 non-null    float64
 7   helpful_votes     854 non-null    float64
 8   rating_image_url  854 non-null    object 
 9   url               854 non-null    object 
 10  text              854 non-null    object 
 11  title             854 non-null    object 
 12  trip_type         853 non-null    object 
 13  travel_date       853 non-null    object 
 14  review_id         854 non-null    float64
dtypes: float64(4), object(11)
memory usage: 106.8+ KB


In [10]:
# Add column: username, user_location_id
# df_new_reviews = df_new_reviews[~df_new_reviews['review_id'].isin(df_reviews['review_id'])]
# new_users=get_user(df_new_reviews)
new_users=get_user(df_new_reviews)

new_users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 854 entries, 0 to 853
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   username          854 non-null    object
 1   user_location_id  854 non-null    object
 2   name              854 non-null    object
 3   avatar_thumbnail  854 non-null    object
 4   avatar_small      854 non-null    object
 5   avatar_medium     854 non-null    object
 6   avatar_large      854 non-null    object
 7   avatar_original   854 non-null    object
dtypes: object(8)
memory usage: 53.5+ KB


In [11]:
df_new_reviews['user_location_id']=new_users['user_location_id']
df_new_reviews['username']=new_users['username']
# df_new_reviews.info()


In [12]:
df_new_reviews.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 854 entries, 0 to 2208
Data columns (total 17 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   user              854 non-null    object 
 1   subratings        854 non-null    object 
 2   owner_response    854 non-null    object 
 3   lang              854 non-null    object 
 4   location_id       854 non-null    float64
 5   published_date    854 non-null    object 
 6   rating            854 non-null    float64
 7   helpful_votes     854 non-null    float64
 8   rating_image_url  854 non-null    object 
 9   url               854 non-null    object 
 10  text              854 non-null    object 
 11  title             854 non-null    object 
 12  trip_type         853 non-null    object 
 13  travel_date       853 non-null    object 
 14  review_id         854 non-null    float64
 15  user_location_id  406 non-null    object 
 16  username          406 non-null    object 
d

In [13]:
# Save new review DataFrame to SQL database
df_new_reviews.to_sql("location_reviews",engine_p,if_exists='append',index=False)

854